In [2]:
import numpy as np
import random
import pandas as pd

In [3]:
NUM_PLAYERS = 3
random.seed(7)  # for reproducible output

# Simulating one game

In [8]:
# Helper functions
def randSelectPlayer(candidates, exclude=None):
    """
    Randomly selects and returns one entry from the list of candidates.
    If `exclude` is specified, that entry is excluded from the list of candidates.
    """
    choices = list(candidates)
    if exclude:
        choices.remove(exclude)
    
    return random.choice(choices)

def give(giver, receiver, playerWealth):
    """
    Simulates the give action:
    The minimum of the giver's and receiver's wealth is transferred from the giver to the receiver.
    Returns the updated `playerWealth` dictionary, which consists of player:wealth key-value pairs.
    """
    v = playerWealth[giver]
    w = playerWealth[receiver]
    amt = min(v, w)
    playerWealth[receiver] += amt
    playerWealth[giver] -= amt
    return playerWealth, amt
    
def getPlayerByWealth(playerWealth, amt):
    """
    Looks up the `playerWealth` dictionary of player:wealth key-value pairs.
    Returns the player whose value matches `amt` (an integer).
    """
    for p, w in playerWealth.items():
        if w == amt:
            return p
        
    return None
    

def checkGameStatus(playerWealth, winAmt, loseAmt=0):
    """
    Checks if there are any winners and losers.
    - playerWealth: a dictionary of player:wealth key-value pairs.
    - winAmt: an integer representing the winning condition.
    - loseAmt: an integer representing the losing condition (default 0).
    Returns a pair: (winner, loser). The entries are None if they do not yet exist.
    """
    winner = getPlayerByWealth(playerWealth, winAmt)
    loser = getPlayerByWealth(playerWealth, loseAmt)        
    
    return winner, loser

def reportGameResult(winner, losers, numRounds):
    """
    Outputs the game result.
    - winner: the player that is the winner.
    - losers: a list of losers in order, from the first to the last loser.
    Note: By the naming convention, the first entry is the actual loser of the entire game.
    """
    # prints losers in order
    for i in range(len(losers)):
        print(f"Loser {i+1}: {losers[i]}")
    print(f"Winner: {winner}")
    print(f"Number of rounds played: {numRounds}")

def reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth):
    """
    Outputs the game progress.
    - numRounds: an integer representing the number of rounds played so far.
    - giver: the player that is chosen to give.
    - receiver: the player that is chosen to receive.
    - transferAmt: an integer representing the amount of money transferred from the giver to the receiver.
    - playerWealth: a dictionary of player:wealth key-value pairs, representing each player's current wealth.
    """
    print(f"Round {numRounds}:")
    print(f"Player {giver} gave Player {receiver} ${transferAmt}")
    print(f"Remaining players' wealth: {playerWealth}\n")

In [9]:
# Main function
def playGame(x, y, z, printProgress=False, printResult=False):
    """
    Plays one game.
    - x, y, z: integers representing the initial wealth of Player 1, 2, 3, respectively.
    - printProgress: if True, prints the game progress.
    - printResult: if True, prints the game's result.
    Returns the winner, (first) loser, and number of rounds until the game ends.
    """
    # initialise
    playerWealth = {1: x, 2: y, 3: z}
    numPlayers = NUM_PLAYERS
    total = x + y + z
    winner = None
    losers = []  # losers in order - might be interested in, e.g. P(Player 2 loses before 1)
    numRounds = 0
    
    while numPlayers > 1:
        giver = randSelectPlayer(playerWealth)
        receiver = randSelectPlayer(playerWealth, exclude=giver)
        playerWealth, transferAmt = give(giver, receiver, playerWealth)
        
        winner, loser = checkGameStatus(playerWealth, total)
        # check if a loser has appeared
        if loser:
            losers.append(loser)
            numPlayers -= 1
            playerWealth.pop(loser)
        
        numRounds += 1
        
        if printProgress:
            reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth)
    
    # sanity checks - at this point, should have 1 winner left, and 2 losers
#     assert(winner)
#     assert(numPlayers == 1)
#     assert(len(losers) == NUM_PLAYERS-1)
    
    if printResult:
        reportGameResult(winner, losers, numRounds)
        
    return winner, losers[0], numRounds

## Examples

In [10]:
#playGame(1,2,3, printProgress=True, printResult=True)

In [11]:
#playGame(12,24,36, printProgress=True, printResult=True)

# Simulating the game multiple times and estimating probabilities

In [12]:
# Helper function
def tallyToDf(winners, losers):
    """
    Counts how many times each player has been the winner and loser, from the given tallies
    (lists of entries 1,2,3, representing the player that has won/lost that game).
    Returns the aggregated result as a DataFrame with rows = (winner, loser), columns = Players (1,2,3)
    """
    winnerCount = [(winners.count(i+1)) for i in range(NUM_PLAYERS)]
    loserCount = [(losers.count(i+1)) for i in range(NUM_PLAYERS)]
    data = {"winner": winnerCount, "loser": loserCount}
    return pd.DataFrame.from_dict(data, orient='index', columns=range(1, NUM_PLAYERS+1))

In [13]:
# Main function
def simGame(n, initState, report=True):
    """
    Plays the betting game `n` times, with the provided initial amounts.
    Prints a summary of the game statistics, include:
    - number of games where each player is the winner and loser;
    - estimated probability of player 1 being the winner and loser;
    - average number of rounds played to end a game.
    """
    x, y, z = initState
    winners = []
    firstLosers = []
    numRoundsPlayed = []
    
    for i in range(n):
        winner, firstLoser, numRounds = playGame(x, y, z)
        winners.append(winner)
        firstLosers.append(firstLoser)
        numRoundsPlayed.append(numRounds)
    
    summaryDf = tallyToDf(winners, firstLosers)
    player1Won = summaryDf.loc['winner', 1]
    player1Lost = summaryDf.loc['loser', 1]
    
    if report:
        print(summaryDf)
        print('\n')

        print(f"P(winner = Player 1): {player1Won} / {n} = {player1Won / n}")
        print(f"P(loser = Player 1): {player1Lost} / {n} = {player1Lost / n}")
        print(f"Average number of rounds to end game: {np.mean(numRoundsPlayed)}")
    
    return player1Lost/n

## Examples

In [16]:
simGame(100000, (5,7,8))

            1      2      3
winner  24878  34968  40154
loser   44127  32190  23683


P(winner = Player 1): 24878 / 100000 = 0.24878
P(loser = Player 1): 44127 / 100000 = 0.44127
Average number of rounds to end game: 3.75026


0.44127